In [1]:
import requests
import json
import base64
import wave
from openai import OpenAI


API_KEY = "sk-XCoZZc1g4tHDcBEH3C2a9Q"
AI_API_BASE = "https://api.thucchien.ai"

usage_url = "https://api.thucchien.ai/key/info"

usage_headers = {
  "accept": "application/json",
  "Authorization": f"Bearer {API_KEY}"
}

response = requests.get(usage_url, headers=usage_headers)

if response.status_code == 200:
    key_info = response.json()
    info = key_info['info']
    print("Key Alias:", info['key_alias'])
    print("Spend: $" + str(info['spend']))
    print("Max Budget:", info.get('max_budget', 'Unlimited'))
    print("Models:", info['models'])
    print("Created:", info['created_at'])
else:
    print("Error:", response.status_code)
    print(response.text)

ModuleNotFoundError: No module named 'requests'

In [16]:
chat_url = f"{AI_API_BASE}/audio/speech"
chat_headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {API_KEY}"
}
data = {
    "model": "gemini-2.5-pro-preview-tts",
    "input": "Xin chào mừng quý vị và các bạn đến với chương trình Việt Nam trong tim tôi. Hôm nay là một chương trình vô cùng đặc biệt nhân dịp kỷ niệm 80 năm thành lập Tổ Quốc",
    "voice": "Zephyr"
}

response = requests.post(chat_url, headers=chat_headers, data=json.dumps(data))

if response.status_code == 200:
    with open('my_generated_audio.mp3', 'wb') as f:
        f.write(response.content)
else:
    print(f"Error: {response.status_code}")
    print(response.text)

In [25]:
AI_API_BASE = "https://api.thucchien.ai"

audio_url = f"{AI_API_BASE}//gemini/v1beta/models/gemini-2.5-pro-preview-tts:generateContent"
audio_headers = {
    "Content-Type": "application/json",
    "x-goog-api-key": API_KEY
}
data = {
    "contents": [{
        "parts": [{
            "text": """
TTS the following speech of 2 Vietnamese TV hosts Phúc (male, northern Vietnamese, 30y) and Thảo (female, southern Vietnamese, 25y):
Phúc: Xin chào mừng quý vị và các bạn đến với chương trình Việt Nam trong tim tôi [medium pause]. Hôm nay là một chương trình vô cùng đặc biệt nhân dịp kỷ niệm 80 năm thành lập Tổ Quốc.
Thảo: Và trong chương trình ngày hôm nay, chúng ta sẽ cùng nhau nhìn lại lịch sử của lá cờ Việt Nam qua từng năm tháng [medium pause], từ thời phong kiến [short pause] cho đến lá cờ đỏ sao vàng thân thương hiện nay.
"""
        }]
    }],
    "generationConfig": {
        "speechConfig": {
            "multiSpeakerVoiceConfig": {
            "speakerVoiceConfigs": [
                {"speaker": "Phúc", "voiceConfig": {"prebuiltVoiceConfig": {"voiceName": "Achird"}}},
                {"speaker": "Thảo", "voiceConfig": {"prebuiltVoiceConfig": {"voiceName": "Leda"}}}
            ]}
        },
        "responseModalities": ["AUDIO"]
    }
}

response = requests.post(audio_url, headers=audio_headers, data=json.dumps(data))

if response.status_code == 200:
    # API trả về âm thanh dưới dạng chuỗi base64, không phải file mp3 trực tiếp.
    try:
        response_json = response.json()
        audio_data_base64 = response_json['candidates'][0]['content']['parts'][0]['inlineData']['data']
        pcm_data = base64.b64decode(audio_data_base64)
        output_filename = 'my_generated_audio.wav'
        with wave.open(output_filename, 'wb') as wf:
            wf.setnchannels(1)  # Mono
            wf.setsampwidth(2)  # 16-bit
            wf.setframerate(24000) # Tần số lấy mẫu 24kHz
            wf.writeframes(pcm_data)
        
        print(f"Tạo tệp âm thanh thành công: {output_filename}")

    except (KeyError, IndexError):
        print("Lỗi: Không thể phân tích phản hồi từ API.")
        print(response.text)
else:
    print(f"Lỗi: {response.status_code}")
    print(response.text)

Tạo tệp âm thanh thành công: my_generated_audio.wav


In [3]:
AI_API_BASE = "https://api.thucchien.ai"

chat_url = f"{AI_API_BASE}/chat/completions"
chat_headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {API_KEY}"
}
data = {
    "model": "gemini-2.5-pro",
    "messages": [
        {
            "role": "system",
            "content": "You are a copywriter. You should write a good script for a short presentation of user's given topic. The presentation is a slideshow, generated by Imagen4, with a Vietnamese voiceover, generated by Gemini TTS."
        },
        {
            "role": "user",
            "content": """Viết nội dung cho một video giới thiệu khoảng 3 phút về ngôn ngữ lập trình Golang. 
            Dự kiến slide gồm khoảng 10 trang, nói về bối cảnh ra đời, tính năng nổi bật, ưu nhược điểm, và lịch sử phát triển cho đến thời điểm hiện tại.
            Đầu tiên, viết 1 prompt dành cho theme của toàn bộ slide, để tôi có thể thêm vào cho mỗi prompt khi sinh từng slide bằng Imagen4.
            Sau đó, bố trí nội dung sẵn thành từng trang, cùng với gợi ý layout của trang đó.
            """
        }
  ]
}

response = requests.post(chat_url, headers=chat_headers, data=json.dumps(data))

if response.status_code == 200:
    result = response.json()
    print(result['choices'][0]['message']['content'])
else:
    print(f"Error: {response.status_code}")
    print(response.text)

Chắc chắn rồi! Dưới đây là nội dung chi tiết cho video giới thiệu về ngôn ngữ lập trình Golang, được tối ưu cho slideshow và voiceover Tiếng Việt.

---

### **Phần 1: Prompt Theme cho Imagen4**

Đây là prompt bạn có thể sử dụng làm "tiền tố" cho mỗi slide để đảm bảo tính nhất quán về mặt hình ảnh.

**(Copy và thêm vào đầu mỗi prompt khi tạo ảnh cho từng slide)**

**Prompt Theme:**
`Minimalist tech presentation slide, clean vector illustration style. Main color palette of Go-lang blue (#00ADD8), white, and charcoal grey. Features abstract representations of code, networks, and concurrency, with a professional and modern aesthetic.`

---

### **Phần 2: Kịch bản chi tiết cho từng Slide (khoảng 3 phút)**

Dưới đây là bố cục 10 slide, bao gồm tiêu đề, gợi ý layout hình ảnh, và nội dung voiceover.

**(Thời gian đọc ước tính: ~15-20 giây mỗi slide)**

---

#### **Slide 1: Trang Tiêu Đề**

*   **Tiêu đề:** **Go: Sức Mạnh Của Sự Đơn Giản**
*   **Gợi ý Layout (Imagen4 Prompt):** `[Prompt Theme] 

In [15]:
import base64

gen_image_url = f"{AI_API_BASE}/images/generations"
headers = {
  "Content-Type": "application/json",
  "Authorization": f"Bearer {API_KEY}"
}

imagen4_prompt = """
A minimalist tech presentation slide in a clean vector illustration style.
The main color palette is Go-lang blue (#00ADD8), white, and charcoal grey.
Except for a small text "ValtecVN" in the top right of the page, in header section, there **must be no text in the page**.
The slide shows portraits of the three creators (Robert Griesemer, Rob Pike, Ken Thompson) on one side. 
On the other side, icons representing problems: a tangled ball of yarn for 'complexity', a slow snail on a CPU for 'slow compilation'.
"""

data = {
  "model": "imagen-4",
  "prompt": imagen4_prompt,
  "n": "4",
  "size": "1536x1024",
}

response = requests.post(gen_image_url, headers=headers, data=json.dumps(data))

if response.status_code == 200:
  result = response.json()
  for i, data in enumerate(result['data']):
    b64_data = data['b64_json']
    image_data = base64.b64decode(b64_data)
    with open(f"slide_2_{i+1}.png", "wb") as f:
      f.write(image_data)
  print("Image saved")
else:
  print(f"Error: {response.status_code}")
  print(response.text)

Error: 400
{"error":{"message":"litellm.BadRequestError: Vertex_aiException BadRequestError - {\n  \"error\": {\n    \"code\": 403,\n    \"message\": \"Permission 'aiplatform.endpoints.predict' denied on resource '//aiplatform.googleapis.com/projects/aithucchien-11-2025/locations/us-central1/publishers/google/models/imagen-4.0-generate-001' (or it may not exist).\",\n    \"status\": \"PERMISSION_DENIED\",\n    \"details\": [\n      {\n        \"@type\": \"type.googleapis.com/google.rpc.ErrorInfo\",\n        \"reason\": \"IAM_PERMISSION_DENIED\",\n        \"domain\": \"aiplatform.googleapis.com\",\n        \"metadata\": {\n          \"resource\": \"projects/aithucchien-11-2025/locations/us-central1/publishers/google/models/imagen-4.0-generate-001\",\n          \"permission\": \"aiplatform.endpoints.predict\"\n        }\n      }\n    ]\n  }\n}\n. Received Model Group=imagen-4\nAvailable Model Group Fallbacks=None","type":null,"param":null,"code":"400"}}


In [ ]:
from openai import OpenAI
import base64

client = OpenAI(
  api_key=API_KEY,
  base_url="https://api.thucchien.ai"
)

flash_image_prompt_1 = """
A minimalist tech presentation slide in a clean vector illustration style.
The main color palette is Go-lang blue (#00ADD8), white, and charcoal grey.
This slide features the Go language Gopher mascot in the center.
The background has abstract digital network lines.
Except for a small text "ValtecVN" in the top right of the page, in header section, there **must be no text in the page**.
However, just leave a clear, empty space as a placeholder area so that I can manually add the text later.
"""

flash_image_prompt_2 = """
A minimalist tech presentation slide in a clean vector illustration style.
The main color palette is Go-lang blue (#00ADD8), white, and charcoal grey.
Except for a small text "ValtecVN" in the top right of the page, in header section, there **must be no text in the page**.
This slide shows portraits of the three creators (Robert Griesemer, Rob Pike, Ken Thompson) on one side. 
On the other side, icons representing problems: a tangled ball of yarn for 'complexity', a slow snail on a CPU for 'slow compilation'.
"""

response = client.chat.completions.create(
  model="gemini-2.5-flash-image-preview",
  messages=[{"role": "user", "content": flash_image_prompt_2}],
  modalities=["image"]
)

base64_string = response.choices[0].message.images[0].get("image_url").get("url")
if ',' in base64_string:
  header, encoded = base64_string.split(',', 1)
else:
  encoded = base64_string
image_data = base64.b64decode(encoded)
with open("slide_2.png", "wb") as f:
  f.write(image_data)
print("Image saved")

Image saved
